In [1]:
import pandas as pd
import numpy as np
import json 
import psycopg2
import matplotlib.pyplot as plt

In [2]:
# connect to the PostgreSQL server
conn = psycopg2.connect(host="imt575group4.cdzfpivjhvbj.us-east-1.rds.amazonaws.com",
	                    database="imt575", user="amir", password="pleasework123")
business = pd.read_sql("SELECT * FROM business WHERE city LIKE '%Las%Vegas%'",conn)

In [3]:
business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,1gU1ZNbcvk0puuUQjbl23w,Sushi Factory,Southwest,"7480 S Rainbow Blvd, Ste C",Las Vegas,NV,89139,36.0527735,-115.2425,4.0,152,1,"{u'Alcohol': u'beer_and_wine', u'HasTV': u'Tru...","Restaurants, Sushi Bars","{u'Monday': u'11:30-0:0', u'Tuesday': u'11:30-..."
1,4E9twWQwae4ELAKUIVbFpw,The Wax Room,Spring Valley,"Studio Salons, 9640 W Tropicana Ave, Room 118",Las Vegas,NV,89147,36.119018261,-115.280227661,5.0,3,0,"{u'BusinessAcceptsCreditCards': u'True', u'Res...","Beauty & Spas, Hair Removal, Waxing","{u'Monday': u'9:30-20:0', u'Tuesday': u'9:30-2..."
2,tZW4NLpEYinu4mIxdKj0Pw,The Rose Grill,Downtown,3328 E Charleston Blvd,Las Vegas,NV,89104,36.1585755,-115.0996395,2.5,7,1,"{u'DogsAllowed': u'False', u'Alcohol': u'full_...","Pizza, Burgers, Restaurants, Salad","{u'Monday': u'10:0-2:0', u'Tuesday': u'10:0-2:..."
3,MLvSu4zlJID1NtxsDLM09A,Charleston Outlet,Downtown,1548 E Charleston Blvd,Las Vegas,NV,89104,36.1585774,-115.1313389,3.0,17,1,"{u'BusinessAcceptsCreditCards': u'True', u'Res...","Men's Clothing, Shopping, Fashion, Thrift Stor...","{u'Monday': u'9:0-21:0', u'Tuesday': u'9:0-21:..."
4,27vCl2nFGa4q-bHSvMTTuQ,A Welding Shop,Downtown,220 W Wyoming Ave,Las Vegas,NV,89102,36.1527005,-115.1586001,3.5,3,1,None,"Local Services, Metal Fabricators","{u'Friday': u'8:30-15:30', u'Tuesday': u'8:30-..."


In [4]:
business.shape

(30430, 15)

In [5]:
business.columns

Index(['business_id', 'name', 'neighborhood', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [6]:
len(pd.unique(business.business_id))

30430

### Working with categories

In [7]:
business_1 = business.copy()

In [8]:
business_1.categories.fillna('None', inplace = True)

In [9]:
high_level_cat = {
'Active Life' :'active',
'Arts & Entertainment' :'arts',
'Automotive': 'auto',
'Beauty & Spas': 'beautysvc',
'Education' :'education',
'Event Planning & Services' :'eventservices',
'Financial Services' :'financialservices',
'Food' : 'restaurants', #food
'Health & Medical' :'health',
'Home Services' :'homeservices',
'Hotels & Travel' :'hotelstravel',
'Local Flavor' :'localflavor',
'Local Services' :'localservices',
'Mass Media' :'massmedia',
'Nightlife' :'nightlife',
'Pets' :'pets',
'Professional Services' : 'professional',
'Public Services & Government' : 'publicservicesgovt',
'Real Estate' :'realestate',
'Religious Organizations' :'religiousorgs',
'Restaurants' :'restaurants',
'Shopping' :'shopping'
}

In [10]:
def filter_category(list_cat):
    new_cat = ''
    for k,v in high_level_cat.items():
        if k in list_cat and v not in new_cat:
            if new_cat != '':
                new_cat=new_cat+','+v
            else:
                new_cat=new_cat+v
    return new_cat

In [11]:
business_1['category'] = business_1.categories.apply(filter_category)

In [12]:
business_1[['category', 'categories']].head(10)

,category,categories
0,restaurants,"Restaurants, Sushi Bars"
1,beautysvc,"Beauty & Spas, Hair Removal, Waxing"
2,restaurants,"Pizza, Burgers, Restaurants, Salad"
3,shopping,"Men's Clothing, Shopping, Fashion, Thrift Stor..."
4,localservices,"Local Services, Metal Fabricators"
5,"eventservices,hotelstravel","Hotels, Event Planning & Services, Hotels & Tr..."
6,restaurants,"Tapas/Small Plates, Steakhouses, Spanish, Rest..."
7,active,"Interval Training Gyms, Gyms, Fitness & Instru..."
8,eventservices,"Party Equipment Rentals, Party Supplies, Event..."
9,localservices,"Community Service/Non-Profit, Local Services"


In [13]:
business_1.drop(['categories'], axis=1, inplace = True)

## Splitting attributes column

In [14]:
business_2 = business_1.copy()

import re
def convert_unicode(x):
    x = x.replace("u\'", "\"")
    x = x.replace("\'", "\"")
    x = x.replace("u\"", "")
    x = x.replace("}\"", "}")
    x = x.replace("True", "\"True\"")
    x = x.replace("False", "\"False\"")
    x = x.replace("\"\"", "\"")
    return x

In [15]:
business_2.attributes = business_2.attributes.apply(convert_unicode)

In [16]:
business_2.attributes.iloc[0]

'{"Alcohol": "beer_and_wine", "HasTV": "True", "NoiseLevel": "average", "RestaurantsAttire": "casual", "BusinessAcceptsCreditCards": "True", "Ambience": {"romantic": "False", "intimate": "False", "classy": "False", "hipster": "False", "divey": "False", "touristy": "False", "trendy": "True", "upscale": "False", "casual": "True"}, "RestaurantsGoodForGroups": "True", "Caters": "True", "WiFi": "free", "RestaurantsReservations": "True", "BusinessAcceptsBitcoin": "False", "RestaurantsTableService": "True", "RestaurantsTakeOut": "True", "GoodForKids": "True", "WheelchairAccessible": "True", "DogsAllowed": "False", "BikeParking": "False", "OutdoorSeating": "False", "RestaurantsPriceRange2": "2", "RestaurantsDelivery": "False", "GoodForMeal": {"dessert": "True", "latenight": "False", "lunch": "True", "dinner": "True", "breakfast": "False", "brunch": "True"}, "BusinessParking": {"garage": "False", "street": "False", "validated": "True", "lot": "False", "valet": "False"}}'

In [17]:
business_2.head(2)

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,hours,category
0,1gU1ZNbcvk0puuUQjbl23w,Sushi Factory,Southwest,"7480 S Rainbow Blvd, Ste C",Las Vegas,NV,89139,36.0527735,-115.2425,4.0,152,1,"{""Alcohol"": ""beer_and_wine"", ""HasTV"": ""True"", ...","{u'Monday': u'11:30-0:0', u'Tuesday': u'11:30-...",restaurants
1,4E9twWQwae4ELAKUIVbFpw,The Wax Room,Spring Valley,"Studio Salons, 9640 W Tropicana Ave, Room 118",Las Vegas,NV,89147,36.119018261,-115.280227661,5.0,3,0,"{""BusinessAcceptsCreditCards"": ""True"", ""Restau...","{u'Monday': u'9:30-20:0', u'Tuesday': u'9:30-2...",beautysvc


In [18]:
len(pd.unique(business_2.business_id))

30430

In [19]:
import ast
def parse_creditcards(x):
    a = ast.literal_eval(x)
    if a:
        return a['BusinessAcceptsCreditCards'] if 'BusinessAcceptsCreditCards' in a else ''
    else:
        return ''

def parse_goodforkids(x):
    a = ast.literal_eval(x)
    if a:
        return a['GoodForKids'] if 'GoodForKids' in a else ''
    else:
        return ''
    
def parse_OutdoorSeating(x):
    a = ast.literal_eval(x)
    if a:
        return a['OutdoorSeating'] if 'OutdoorSeating' in a else ''
    else:
        return ''
    
def parse_BusinessParking(x):
    a = ast.literal_eval(x)
    if a:
        return str(a['BusinessParking']) if 'BusinessParking' in a else ''
    else:
        return ''
    
def parse_RestaurantsDelivery(x):
    a = ast.literal_eval(x)
    if a:
        return a['RestaurantsDelivery'] if 'RestaurantsDelivery' in a else ''
    else:
        return ''
    
def parse_RestaurantsPriceRange(x):
    a = ast.literal_eval(x)
    if a:
        return int(a['RestaurantsPriceRange2']) if 'RestaurantsPriceRange2' in a else ''
    else:
        return None
    
def parse_RestaurantsReservations(x):
    a = ast.literal_eval(x)
    if a:
        return a['RestaurantsReservations'] if 'RestaurantsReservations' in a else ''
    else:
        return ''
    
def parse_RestaurantsTakeOut(x):
    a = ast.literal_eval(x)
    if a:
        return a['RestaurantsTakeOut'] if 'RestaurantsTakeOut' in a else ''
    else:
        return ''

In [20]:
## test
business_2['attributes_BusinessAcceptsCreditCards'] = business_2['attributes'].apply(parse_creditcards)
business_2['attributes_GoodForKids'] = business_2['attributes'].apply(parse_goodforkids)
business_2['attributes_OutdoorSeating'] = business_2['attributes'].apply(parse_OutdoorSeating)
business_2['attributes_BusinessParking'] = business_2['attributes'].apply(parse_BusinessParking)
business_2['attributes_RestaurantsDelivery'] = business_2['attributes'].apply(parse_RestaurantsDelivery)
business_2['attributes_RestaurantsPriceRange2'] = business_2['attributes'].apply(parse_RestaurantsPriceRange)
business_2['attributes_RestaurantsReservations'] = business_2['attributes'].apply(parse_RestaurantsReservations)
business_2['attributes_RestaurantsTakeOut'] = business_2['attributes'].apply(parse_RestaurantsTakeOut)

In [21]:
business_2.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,hours,category,attributes_BusinessAcceptsCreditCards,attributes_GoodForKids,attributes_OutdoorSeating,attributes_BusinessParking,attributes_RestaurantsDelivery,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_RestaurantsTakeOut
0,1gU1ZNbcvk0puuUQjbl23w,Sushi Factory,Southwest,"7480 S Rainbow Blvd, Ste C",Las Vegas,NV,89139,36.0527735,-115.2425,4.0,...,"{u'Monday': u'11:30-0:0', u'Tuesday': u'11:30-...",restaurants,True,True,False,"{'garage': 'False', 'street': 'False', 'valida...",False,2,True,True
1,4E9twWQwae4ELAKUIVbFpw,The Wax Room,Spring Valley,"Studio Salons, 9640 W Tropicana Ave, Room 118",Las Vegas,NV,89147,36.119018261,-115.280227661,5.0,...,"{u'Monday': u'9:30-20:0', u'Tuesday': u'9:30-2...",beautysvc,True,,,"{'garage': 'False', 'street': 'False', 'valida...",,1,,
2,tZW4NLpEYinu4mIxdKj0Pw,The Rose Grill,Downtown,3328 E Charleston Blvd,Las Vegas,NV,89104,36.1585755,-115.0996395,2.5,...,"{u'Monday': u'10:0-2:0', u'Tuesday': u'10:0-2:...",restaurants,True,,False,"{'garage': 'False', 'street': 'False', 'valida...",,,False,True
3,MLvSu4zlJID1NtxsDLM09A,Charleston Outlet,Downtown,1548 E Charleston Blvd,Las Vegas,NV,89104,36.1585774,-115.1313389,3.0,...,"{u'Monday': u'9:0-21:0', u'Tuesday': u'9:0-21:...",shopping,True,,,"{'garage': 'False', 'street': 'True', 'validat...",,1,,
4,27vCl2nFGa4q-bHSvMTTuQ,A Welding Shop,Downtown,220 W Wyoming Ave,Las Vegas,NV,89102,36.1527005,-115.1586001,3.5,...,"{u'Friday': u'8:30-15:30', u'Tuesday': u'8:30-...",localservices,,,,,,None,,


In [22]:
business_2[business_2.business_id == 'tZW4NLpEYinu4mIxdKj0Pw']

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,hours,category,attributes_BusinessAcceptsCreditCards,attributes_GoodForKids,attributes_OutdoorSeating,attributes_BusinessParking,attributes_RestaurantsDelivery,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_RestaurantsTakeOut
2,tZW4NLpEYinu4mIxdKj0Pw,The Rose Grill,Downtown,3328 E Charleston Blvd,Las Vegas,NV,89104,36.1585755,-115.0996395,2.5,...,"{u'Monday': u'10:0-2:0', u'Tuesday': u'10:0-2:...",restaurants,True,,False,"{'garage': 'False', 'street': 'False', 'valida...",,,False,True


In [23]:
business_2.attributes_RestaurantsPriceRange2.value_counts(dropna=False)

       10662
2       8411
1       5142
NaN     4389
3       1346
4        480
Name: attributes_RestaurantsPriceRange2, dtype: int64

In [24]:
business_2.attributes_GoodForKids.value_counts(dropna=False)

         21562
True      7043
False     1825
Name: attributes_GoodForKids, dtype: int64

In [25]:
business_2.attributes_RestaurantsReservations.value_counts(dropna=False)

         24133
False     4296
True      2001
Name: attributes_RestaurantsReservations, dtype: int64

In [26]:
business_2.attributes_BusinessAcceptsCreditCards.value_counts(dropna=False)

True     22524
          6454
False     1452
Name: attributes_BusinessAcceptsCreditCards, dtype: int64

In [27]:
len(pd.unique(business_2.business_id))

30430

## Dealing with Nulls

In [28]:
df_rest_3 = business_2.copy()

In [29]:
len(pd.unique(df_rest_3.business_id))

30430

In [30]:
rest_null = pd.DataFrame(df_rest_3.isnull().sum(), columns = ['missing_sum'])

In [31]:
#rest_null.sort_values(by='missing_sum')

In [32]:
# Keeping the ones with less number of nulls
df_rest_3 = df_rest_3[['business_id', 'city', 'review_count', 'stars', 'state',
       'category','attributes_BusinessAcceptsCreditCards',
       'attributes_GoodForKids','attributes_OutdoorSeating', 'attributes_BusinessParking',
        'attributes_RestaurantsDelivery', 'attributes_RestaurantsPriceRange2',
       'attributes_RestaurantsReservations', 'attributes_RestaurantsTakeOut', 'neighborhood', 'latitude', 'longitude'
        ]]

In [33]:
df_rest_3.attributes_BusinessAcceptsCreditCards.value_counts()

True     22524
          6454
False     1452
Name: attributes_BusinessAcceptsCreditCards, dtype: int64

In [34]:
df_rest_3.attributes_GoodForKids.value_counts()

         21562
True      7043
False     1825
Name: attributes_GoodForKids, dtype: int64

In [35]:
df_rest_3.attributes_OutdoorSeating.value_counts()

         23413
False     5080
True      1937
Name: attributes_OutdoorSeating, dtype: int64

In [36]:
df_rest_3.attributes_RestaurantsDelivery.value_counts()

         24229
False     4668
True      1533
Name: attributes_RestaurantsDelivery, dtype: int64

In [37]:
df_rest_3.attributes_RestaurantsPriceRange2.value_counts()

     10662
2     8411
1     5142
3     1346
4      480
Name: attributes_RestaurantsPriceRange2, dtype: int64

In [38]:
df_rest_3.attributes_RestaurantsReservations.value_counts()

         24133
False     4296
True      2001
Name: attributes_RestaurantsReservations, dtype: int64

In [39]:
df_rest_3.attributes_RestaurantsTakeOut.value_counts()

         23090
True      6505
False      835
Name: attributes_RestaurantsTakeOut, dtype: int64

In [40]:
df_rest_3.attributes_BusinessAcceptsCreditCards = df_rest_3.attributes_BusinessAcceptsCreditCards.map({'True':1, 'False':0})
df_rest_3.attributes_GoodForKids = df_rest_3.attributes_GoodForKids.map({'True':1, 'False':0})
df_rest_3.attributes_OutdoorSeating = df_rest_3.attributes_OutdoorSeating.map({'True':1, 'False':0})
#rest.attributes_BusinessParking = rest.attributes_BusinessParking.map({True:1, False:0})
df_rest_3.attributes_RestaurantsDelivery = df_rest_3.attributes_RestaurantsDelivery.map({'True':1, 'False':0})
df_rest_3.attributes_RestaurantsPriceRange2 = df_rest_3.attributes_RestaurantsPriceRange2.map({1:1, 2:2,3:3,4:4})
df_rest_3.attributes_RestaurantsReservations = df_rest_3.attributes_RestaurantsReservations.map({'True':1, 'False':0})
df_rest_3.attributes_RestaurantsTakeOut = df_rest_3.attributes_RestaurantsTakeOut.map({'True':1, 'False':0})

In [41]:
df_rest_3.head()

,business_id,city,review_count,stars,state,category,attributes_BusinessAcceptsCreditCards,attributes_GoodForKids,attributes_OutdoorSeating,attributes_BusinessParking,attributes_RestaurantsDelivery,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_RestaurantsTakeOut,neighborhood,latitude,longitude
0,1gU1ZNbcvk0puuUQjbl23w,Las Vegas,152,4.0,NV,restaurants,1.0,1.0,0.0,"{'garage': 'False', 'street': 'False', 'valida...",0.0,2.0,1.0,1.0,Southwest,36.0527735,-115.2425
1,4E9twWQwae4ELAKUIVbFpw,Las Vegas,3,5.0,NV,beautysvc,1.0,NaN,NaN,"{'garage': 'False', 'street': 'False', 'valida...",NaN,1.0,NaN,NaN,Spring Valley,36.119018261,-115.280227661
2,tZW4NLpEYinu4mIxdKj0Pw,Las Vegas,7,2.5,NV,restaurants,1.0,NaN,0.0,"{'garage': 'False', 'street': 'False', 'valida...",NaN,NaN,0.0,1.0,Downtown,36.1585755,-115.0996395
3,MLvSu4zlJID1NtxsDLM09A,Las Vegas,17,3.0,NV,shopping,1.0,NaN,NaN,"{'garage': 'False', 'street': 'True', 'validat...",NaN,1.0,NaN,NaN,Downtown,36.1585774,-115.1313389
4,27vCl2nFGa4q-bHSvMTTuQ,Las Vegas,3,3.5,NV,localservices,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,Downtown,36.1527005,-115.1586001


In [42]:
df_rest_3.shape

(30430, 17)

In [43]:
len(pd.unique(df_rest_3.business_id))

30430

In [44]:
#df_rest_2['attributes_RestaurantsPriceRange2'].values

In [45]:
df_rest_3.attributes_BusinessAcceptsCreditCards.fillna(round(df_rest_3.attributes_BusinessAcceptsCreditCards.mean(),2), inplace = True)
df_rest_3.attributes_GoodForKids.fillna(round(df_rest_3.attributes_GoodForKids.mean(),2), inplace = True)
df_rest_3.attributes_OutdoorSeating.fillna(round(df_rest_3.attributes_OutdoorSeating.mean(),2), inplace = True)
#rest.attributes_BusinessParking = rest.attributes_BusinessParking.map({True:1, False:0})
df_rest_3.attributes_RestaurantsDelivery.fillna(round(df_rest_3.attributes_RestaurantsDelivery.mean(),2), inplace = True)
df_rest_3.attributes_RestaurantsPriceRange2.fillna(round(df_rest_3.attributes_RestaurantsPriceRange2.mean(),2), inplace = True)
df_rest_3.attributes_RestaurantsReservations.fillna(round(df_rest_3.attributes_RestaurantsReservations.mean(),2), inplace = True)
df_rest_3.attributes_RestaurantsTakeOut.fillna(round(df_rest_3.attributes_RestaurantsTakeOut.mean(),2), inplace = True)

In [46]:
df_rest_3.head()

,business_id,city,review_count,stars,state,category,attributes_BusinessAcceptsCreditCards,attributes_GoodForKids,attributes_OutdoorSeating,attributes_BusinessParking,attributes_RestaurantsDelivery,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_RestaurantsTakeOut,neighborhood,latitude,longitude
0,1gU1ZNbcvk0puuUQjbl23w,Las Vegas,152,4.0,NV,restaurants,1.00,1.00,0.00,"{'garage': 'False', 'street': 'False', 'valida...",0.00,2.00,1.00,1.00,Southwest,36.0527735,-115.2425
1,4E9twWQwae4ELAKUIVbFpw,Las Vegas,3,5.0,NV,beautysvc,1.00,0.79,0.28,"{'garage': 'False', 'street': 'False', 'valida...",0.25,1.00,0.32,0.89,Spring Valley,36.119018261,-115.280227661
2,tZW4NLpEYinu4mIxdKj0Pw,Las Vegas,7,2.5,NV,restaurants,1.00,0.79,0.00,"{'garage': 'False', 'street': 'False', 'valida...",0.25,1.82,0.00,1.00,Downtown,36.1585755,-115.0996395
3,MLvSu4zlJID1NtxsDLM09A,Las Vegas,17,3.0,NV,shopping,1.00,0.79,0.28,"{'garage': 'False', 'street': 'True', 'validat...",0.25,1.00,0.32,0.89,Downtown,36.1585774,-115.1313389
4,27vCl2nFGa4q-bHSvMTTuQ,Las Vegas,3,3.5,NV,localservices,0.94,0.79,0.28,,0.25,1.82,0.32,0.89,Downtown,36.1527005,-115.1586001


## Business Parking Attribute

In [47]:
df_rest_3.attributes_BusinessParking = df_rest_3.attributes_BusinessParking.fillna('None')

In [48]:
df_rest_3.attributes_BusinessParking.head(10)

0    {'garage': 'False', 'street': 'False', 'valida...
1    {'garage': 'False', 'street': 'False', 'valida...
2    {'garage': 'False', 'street': 'False', 'valida...
3    {'garage': 'False', 'street': 'True', 'validat...
4                                                     
5                                                     
6    {'garage': 'True', 'street': 'False', 'validat...
7    {'garage': 'False', 'street': 'False', 'valida...
8                                                     
9                                                     
Name: attributes_BusinessParking, dtype: object

In [49]:
#vegas['Parking'] = 'False'

In [50]:
def attr_func(i):
    if i!='':
        i = ast.literal_eval(i)
        for k,v in i.items():
            if v== 'True':
                return 1
        else:
            return 0

In [51]:
df_rest_3['BusinessParking'] = df_rest_3['attributes_BusinessParking'].apply(attr_func)

In [52]:
df_rest_3['BusinessParking'].head(10)

0    1.0
1    0.0
2    1.0
3    1.0
4    NaN
5    NaN
6    1.0
7    0.0
8    NaN
9    NaN
Name: BusinessParking, dtype: float64

In [53]:
df_rest_3.BusinessParking.fillna(round(df_rest_3.BusinessParking.mean(),2), inplace = True)

In [54]:
df_rest_3.drop(['attributes_BusinessParking'], axis=1, inplace = True)

In [55]:
df_rest_3.head()

,business_id,city,review_count,stars,state,category,attributes_BusinessAcceptsCreditCards,attributes_GoodForKids,attributes_OutdoorSeating,attributes_RestaurantsDelivery,attributes_RestaurantsPriceRange2,attributes_RestaurantsReservations,attributes_RestaurantsTakeOut,neighborhood,latitude,longitude,BusinessParking
0,1gU1ZNbcvk0puuUQjbl23w,Las Vegas,152,4.0,NV,restaurants,1.00,1.00,0.00,0.00,2.00,1.00,1.00,Southwest,36.0527735,-115.2425,1.00
1,4E9twWQwae4ELAKUIVbFpw,Las Vegas,3,5.0,NV,beautysvc,1.00,0.79,0.28,0.25,1.00,0.32,0.89,Spring Valley,36.119018261,-115.280227661,0.00
2,tZW4NLpEYinu4mIxdKj0Pw,Las Vegas,7,2.5,NV,restaurants,1.00,0.79,0.00,0.25,1.82,0.00,1.00,Downtown,36.1585755,-115.0996395,1.00
3,MLvSu4zlJID1NtxsDLM09A,Las Vegas,17,3.0,NV,shopping,1.00,0.79,0.28,0.25,1.00,0.32,0.89,Downtown,36.1585774,-115.1313389,1.00
4,27vCl2nFGa4q-bHSvMTTuQ,Las Vegas,3,3.5,NV,localservices,0.94,0.79,0.28,0.25,1.82,0.32,0.89,Downtown,36.1527005,-115.1586001,0.64


In [56]:
df_rest_3.shape

(30430, 17)

In [57]:
len(pd.unique(df_rest_3.business_id))

30430

In [58]:
df_rest_3.columns

Index(['business_id', 'city', 'review_count', 'stars', 'state', 'category',
       'attributes_BusinessAcceptsCreditCards', 'attributes_GoodForKids',
       'attributes_OutdoorSeating', 'attributes_RestaurantsDelivery',
       'attributes_RestaurantsPriceRange2',
       'attributes_RestaurantsReservations', 'attributes_RestaurantsTakeOut',
       'neighborhood', 'latitude', 'longitude', 'BusinessParking'],
      dtype='object')

In [59]:
df_rest_3.columns = ['business_id', 'city', 'review_count', 'stars', 'state',
       'category','accepts_credit_cards',
       'good_for_kids','outdoor_seating',
        'restaurants_delivery', 'restaurants_price_range',
       'restaurants_reservations',  'restaurants_takeout','neighborhood','latitude', 'longitude','business_parking']

In [60]:
df_rest_3.head()

,business_id,city,review_count,stars,state,category,accepts_credit_cards,good_for_kids,outdoor_seating,restaurants_delivery,restaurants_price_range,restaurants_reservations,restaurants_takeout,neighborhood,latitude,longitude,business_parking
0,1gU1ZNbcvk0puuUQjbl23w,Las Vegas,152,4.0,NV,restaurants,1.00,1.00,0.00,0.00,2.00,1.00,1.00,Southwest,36.0527735,-115.2425,1.00
1,4E9twWQwae4ELAKUIVbFpw,Las Vegas,3,5.0,NV,beautysvc,1.00,0.79,0.28,0.25,1.00,0.32,0.89,Spring Valley,36.119018261,-115.280227661,0.00
2,tZW4NLpEYinu4mIxdKj0Pw,Las Vegas,7,2.5,NV,restaurants,1.00,0.79,0.00,0.25,1.82,0.00,1.00,Downtown,36.1585755,-115.0996395,1.00
3,MLvSu4zlJID1NtxsDLM09A,Las Vegas,17,3.0,NV,shopping,1.00,0.79,0.28,0.25,1.00,0.32,0.89,Downtown,36.1585774,-115.1313389,1.00
4,27vCl2nFGa4q-bHSvMTTuQ,Las Vegas,3,3.5,NV,localservices,0.94,0.79,0.28,0.25,1.82,0.32,0.89,Downtown,36.1527005,-115.1586001,0.64


In [61]:
df_rest_3.reset_index(level=None, inplace = True)

In [62]:
df_rest_3.drop(['index'], axis=1, inplace = True)

In [63]:
len(pd.unique(df_rest_3.business_id))

30430

In [69]:
df_rest_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30430 entries, 0 to 30429
Data columns (total 17 columns):
business_id                 30430 non-null object
city                        30430 non-null object
review_count                30430 non-null int64
stars                       30430 non-null float64
state                       30430 non-null object
category                    30430 non-null object
accepts_credit_cards        30430 non-null float64
good_for_kids               30430 non-null float64
outdoor_seating             30430 non-null float64
restaurants_delivery        30430 non-null float64
restaurants_price_range     30430 non-null float64
restaurants_reservations    30430 non-null float64
restaurants_takeout         30430 non-null float64
neighborhood                30430 non-null object
latitude                    30430 non-null object
longitude                   30430 non-null object
business_parking            30430 non-null float64
dtypes: float64(9), int64(1), obj

In [ ]:
df_rest_3.to_csv('data/all_business_final.csv')

## Create new features

In [3]:
df_business = pd.read_sql("SELECT * FROM all_business_vegas",conn)

In [4]:
df_business = df_business[df_business.category!='']
df_business.shape

(30341, 24)

In [5]:
pd.unique(df_business.cat)

array(['active', 'shopping', 'massmedia', 'beautysvc', 'homeservices',
       'eventservices', 'auto', 'restaurants', 'localservices',
       'financialservices', 'health', 'pets', 'arts', 'nightlife',
       'hotelstravel', 'education', 'professional', 'publicservicesgovt',
       'religiousorgs', 'localflavor'], dtype=object)

In [6]:
def create_features(feature):
    dict_neigh = {}
    for neigh in pd.unique(df_business.neighborhood.values):
        df_business_neigh = df_business[df_business.neighborhood == neigh]
        dict_neigh[neigh]=df_business_neigh[df_business_neigh.category.str.contains(feature)].shape[0]
    
    print("###################"+feature+"############################")
    print(dict_neigh)
    df_business[feature+'_count'] = df_business.neighborhood.apply(lambda x:dict_neigh[x])

In [7]:
create_features('hotelstravel')

###################hotelstravel############################
{'Centennial': 12, 'Westside': 56, 'Eastside': 105, 'Spring Valley': 36, 'Downtown': 154, 'Chinatown': 108, 'Southeast': 170, 'The Strip': 324, 'South Summerlin': 20, 'Southwest': 27, 'Northwest': 25, 'Sunrise': 58, 'University': 9, 'The Lakes': 1, 'Summerlin': 11, 'Anthem': 3, 'North Valley': 0}


In [8]:
create_features('education')

###################education############################
{'Centennial': 28, 'Westside': 89, 'Eastside': 30, 'Spring Valley': 65, 'Downtown': 51, 'Chinatown': 38, 'Southeast': 71, 'The Strip': 37, 'South Summerlin': 14, 'Southwest': 33, 'Northwest': 51, 'Sunrise': 24, 'University': 14, 'The Lakes': 7, 'Summerlin': 25, 'Anthem': 4, 'North Valley': 0}


In [9]:
create_features('shopping')

###################shopping############################
{'Centennial': 147, 'Westside': 818, 'Eastside': 269, 'Spring Valley': 367, 'Downtown': 628, 'Chinatown': 349, 'Southeast': 601, 'The Strip': 974, 'South Summerlin': 101, 'Southwest': 179, 'Northwest': 297, 'Sunrise': 195, 'University': 43, 'The Lakes': 13, 'Summerlin': 85, 'Anthem': 48, 'North Valley': 0}


In [10]:
create_features('massmedia')

###################massmedia############################
{'Centennial': 1, 'Westside': 13, 'Eastside': 6, 'Spring Valley': 6, 'Downtown': 9, 'Chinatown': 9, 'Southeast': 7, 'The Strip': 12, 'South Summerlin': 1, 'Southwest': 1, 'Northwest': 2, 'Sunrise': 0, 'University': 1, 'The Lakes': 1, 'Summerlin': 1, 'Anthem': 0, 'North Valley': 0}


In [11]:
create_features('arts')

###################arts############################
{'Centennial': 15, 'Westside': 113, 'Eastside': 119, 'Spring Valley': 64, 'Downtown': 236, 'Chinatown': 83, 'Southeast': 109, 'The Strip': 559, 'South Summerlin': 26, 'Southwest': 38, 'Northwest': 51, 'Sunrise': 33, 'University': 22, 'The Lakes': 3, 'Summerlin': 19, 'Anthem': 7, 'North Valley': 0}


In [12]:
create_features('active')

###################active############################
{'Centennial': 57, 'Westside': 201, 'Eastside': 54, 'Spring Valley': 168, 'Downtown': 109, 'Chinatown': 90, 'Southeast': 157, 'The Strip': 182, 'South Summerlin': 76, 'Southwest': 84, 'Northwest': 133, 'Sunrise': 47, 'University': 8, 'The Lakes': 8, 'Summerlin': 74, 'Anthem': 12, 'North Valley': 1}


In [13]:
df_business.head()

,business_id,city,review_count,stars,state,category,accepts_credit_cards,good_for_kids,outdoor_seating,restaurants_delivery,...,business_parking,cat,hotelstravel_count,auto_count,restaurants_count,massmedia_count,arts_count,active_count,education_count,shopping_count
0,-zsZLjeVBh0SpWlOXAbQaw,Las Vegas,8.0,3.0,NV,"active,homeservices,shopping",1.00,0.79,0.28,0.25,...,0.00,active,12,68.0,238.0,1,15,57,28,147
1,-zR_UC4wsDc4MgKRJZ8XSg,Las Vegas,3.0,3.5,NV,shopping,1.00,0.79,0.28,0.25,...,1.00,shopping,56,336.0,982.0,13,113,201,89,818
2,-zqi1B7UwCtpoZh_qZIqHw,Las Vegas,6.0,4.0,NV,massmedia,0.94,0.79,0.28,0.25,...,0.64,massmedia,105,110.0,662.0,6,119,54,30,269
3,-ZNTxbUVRg25LhOxEuW4rQ,Las Vegas,313.0,4.0,NV,beautysvc,1.00,0.79,0.28,0.25,...,1.00,beautysvc,36,170.0,684.0,6,64,168,65,367
4,-znjWwaNajYumGuybVu-dw,Las Vegas,15.0,4.5,NV,homeservices,1.00,0.79,0.28,0.25,...,0.64,homeservices,154,316.0,923.0,9,236,109,51,628


In [14]:
df_business.to_csv('data/all_business_newfeatures.csv')